In [1]:
import numpy as np
from scipy import stats

In [2]:
# import dataset
from sklearn import datasets

In [3]:
X, y = datasets.load_iris(return_X_y=True)
X, y

(array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
        [5

In [87]:
class BayesClassifier:
    '''BayesClassifer takes in a numpy array like list
    of a X: featurs and y: target. 
    
    The X and y array must be of the same length'''
    
    def __init__(self):
        pass

    
    def separate_classes(self, X, y):
        '''
        Takes the feature and target arrays, and seperates them in to
        a subsets of data by the unique classes. This is a helper function
        That will be used in the fit method of the class. 
        '''
       
        self.X = X
        self.y = y
        self.classes = np.unique(y)
        
        # we need the frequencies of the classes.
        # these frequencies will be used in the predicting the 
        # target class of an unobserved datapoint.
        class_type, num_of_class = np.unique(y, return_counts=True)
        self.class_counts = dict(zip(class_type, num_of_class))
        self.class_frequencies = {}
        # frequencies currently is just the total count. We want the proportions. 
        for key, value in self.class_counts.items():
            self.class_frequencies[key] = self.class_counts[key]/sum(self.class_counts.values())
        
        # need to be able to get a subset of arrays by class type. 
        # we will store these in a dictionary. The keys being the class type
        # and the values will be each feature array that has the class as its corresponding target
        
        class_indexes = {}
        subsets = {}
        for i in set(y):
            class_indexes[i] = np.argwhere(y==i)
            subsets[i] = X[class_indexes[i]]
        
        return subsets
    
    def fit(self, X, y):
        '''fit function. Fit the function to the training set.
        takes in an X feature array, and a y Target vector. 
        '''
        
        # to fit the data we need to get the means and the standard deviations for 
        # each class. Will call the separate_classes method to get the unique feature
        # arrays for each class. Then we will get the means and standard deviations
        # for each feature by class. 
        subsetted_X = self.separate_classes(X, y)
        
        self.class_means = {}
        self.class_standard_dev = {}
        for i in subsetted_X:
            self.class_means[i] = subsetted_X[i].mean(axis=0)
            self.class_standard_dev[i] = subsetted_X[i].std(axis=0)
    
    def probability_density_func(self, class_index, x):
        '''
        HT https://en.wikipedia.org/wiki/Gaussian_function
        
        Use the probability function to calculate the relative liklihood that a value
        of our random variable 'x' would equal our sample.
        
        
        '''
        
        class_mean = self.class_means[class_index]
        class_stdev = self.class_standard_dev[class_index]
        
        exponent = np.exp(-((x - class_mean)**2 / (2 * class_stdev**2)))
        base = 1 / (class_stdev * np.sqrt(2 * np.pi))
        return (base * exponent)
    

        
    def predicted_probabilities(self, x):
        '''Get a list with the probabilities for each class and return the class
        with the highest probablity. This will make our output a class, not a probability'''
        
        pred_probabilites = []
                       
        for idx, cls in enumerate(self.classes):
            cls_frequency = np.log(self.class_frequencies[cls])
            pred_proba = np.sum(np.log(self.probability_density_func(idx, x)))
            pred_proba = pred_proba + cls_frequency
            pred_probabilites.append(pred_proba)
        
        return self.classes[np.argmax(pred_probabilites)]

    def predict(self, X):
        '''Get the predicted class for each target x in the given X dataset'''
        y_pred = [self.predicted_probabilities(x) for x in X]
        


In [88]:
model = BayesClassifier()
model.fit(X, y)
predictions = model.predict(X)

In [89]:
print(predictions)

None


In [90]:
# instantiate the BayesClassifier
classifier = BayesClassifier()

In [91]:
# separate_classes method. This is the helper function
# that is used in the fit method of the Class
# This shows the feature arrays that correspond with a
# given target class
classifier.separate_classes(X, y)

{0: array([[[5.1, 3.5, 1.4, 0.2]],
 
        [[4.9, 3. , 1.4, 0.2]],
 
        [[4.7, 3.2, 1.3, 0.2]],
 
        [[4.6, 3.1, 1.5, 0.2]],
 
        [[5. , 3.6, 1.4, 0.2]],
 
        [[5.4, 3.9, 1.7, 0.4]],
 
        [[4.6, 3.4, 1.4, 0.3]],
 
        [[5. , 3.4, 1.5, 0.2]],
 
        [[4.4, 2.9, 1.4, 0.2]],
 
        [[4.9, 3.1, 1.5, 0.1]],
 
        [[5.4, 3.7, 1.5, 0.2]],
 
        [[4.8, 3.4, 1.6, 0.2]],
 
        [[4.8, 3. , 1.4, 0.1]],
 
        [[4.3, 3. , 1.1, 0.1]],
 
        [[5.8, 4. , 1.2, 0.2]],
 
        [[5.7, 4.4, 1.5, 0.4]],
 
        [[5.4, 3.9, 1.3, 0.4]],
 
        [[5.1, 3.5, 1.4, 0.3]],
 
        [[5.7, 3.8, 1.7, 0.3]],
 
        [[5.1, 3.8, 1.5, 0.3]],
 
        [[5.4, 3.4, 1.7, 0.2]],
 
        [[5.1, 3.7, 1.5, 0.4]],
 
        [[4.6, 3.6, 1. , 0.2]],
 
        [[5.1, 3.3, 1.7, 0.5]],
 
        [[4.8, 3.4, 1.9, 0.2]],
 
        [[5. , 3. , 1.6, 0.2]],
 
        [[5. , 3.4, 1.6, 0.4]],
 
        [[5.2, 3.5, 1.5, 0.2]],
 
        [[5.2, 3.4, 1.4, 0.2]],
 
        [[4

In [92]:
# class_frequencies attribute shows how frequently the 
# a given class appears in the dataset. 
# this will be used when predicting the outcome of an
# unobserved data point.
classifier.class_frequencies

{0: 0.3333333333333333, 1: 0.3333333333333333, 2: 0.3333333333333333}

In [97]:
classifier.fit(X, y)
print(classifier.class_means)
print(classifier.class_standard_dev)
print(classifier.classes)

{0: array([[5.006, 3.428, 1.462, 0.246]]), 1: array([[5.936, 2.77 , 4.26 , 1.326]]), 2: array([[6.588, 2.974, 5.552, 2.026]])}
{0: array([[0.34894699, 0.37525458, 0.17191859, 0.10432641]]), 1: array([[0.51098337, 0.31064449, 0.46518813, 0.19576517]]), 2: array([[0.62948868, 0.31925538, 0.54634787, 0.27188968]])}
[0 1 2]


In [98]:
prediction = classifier.predict(X[0])

/home/tmbern/.local/share/virtualenvs/CS-Data-Science-Build-Week-1-8zfipYz8/lib/python3.6/site-packages/ipykernel_launcher.py:88: RuntimeWarning: divide by zero encountered in log


In [99]:
X[0]

array([5.1, 3.5, 1.4, 0.2])

In [55]:
predicitions = classifier.predict(X)

AttributeError: 'BayesClassifier' object has no attribute 'predicted_probablities'

In [15]:
np.exp(1)

2.718281828459045

In [19]:
np.pi

3.141592653589793

In [27]:
np.sqrt(4)

2.0